In [1]:
import biosteam as bst
import thermosteam as tmo
import biorefineries as bf
from biorefineries import cellulosic
from lignin_saf.reactor import *

In [2]:
tmo.Stream.display_units.N = 20       # Increasing number of display units to see all components of streams 
tmo.MultiStream.display_units.N = 20  

In [3]:
def create_chemicals():


    # copying cellulosic ethanol chemicals 
    chems = cellulosic.create_cellulosic_ethanol_chemicals().copy()

    # Adding pre-existing chemicals
    methanol = tmo.Chemical('Methanol')
    hydrogen = tmo.Chemical('Hydrogen')
    methane = tmo.Chemical('Methane')
    activated_carbon = tmo.Chemical('ActivatedCarbon', search_db=False, default=True, phase='s')

    
    # Custom chemical properties estimated using NIST ThermoDataEngine (TDE) from Aspen Plus V14
    # Only propylguaiacol was native to Aspen, every other component was user defined
    # All six of these user defined components are the same as chosen by Bartling et al Fig S8
    # Constant properties like normal boiling point, critical temp, critical pressure were added
    # Custom models for heat capacity, density as functions of temperatures were NOT included
   
    propylguaiacol = tmo.Chemical(  # S-Lignin Monomer 
        'Propylguaiacol',
        default = True,      # Defaults all other properties 
        search_db=False,     # Since not present in database, do not search
        formula='C10H14O2',  # Chemical formulae
        phase='l',           # phase at rtp
        omega = 0.6411,      # accentric factor
        Tb = 541.7,          # [K]  normal boiling point
        # Tc = 749,            # [K]  critical temperature
        # Pc = 2.9e6,          # [Pa] critical pressure
        Hvap = 7.78e4,       # [J/mol] enthalpy of vaporization at 298 K
        rho = 1056.3,        # [kg/m3] density at rtp

    )
    propylguaiacol.synonyms = ('4-Propylguaiacol',) # Synonyms that can be used to refer to it

    
    propylsyringol = tmo.Chemical(   # G-Lignin Monomer 
        'Propylsyringol',
        default = True,
        search_db=False,
        formula='C11H16O3',
        phase='l',
        omega = 0.87334, 
        Tb = 617.3,      
        #Tc = 819,        
        # Pc = 2565348.3,   
        Hvap = 1.07e5,    
        rho  = 1274.3    
    )
    propylsyringol.synonyms = ('4-Propylsyringol',)

    syringaresinol = tmo.Chemical(  # S-Lignin Dimer 
        'Syringaresinol',
        default = True,
        search_db = False,
        formula = 'C22H26O8',
        phase = 'l',
        omega = 1.4608,
        Tb = 796.7,
        # Tc = 979,
        # Pc = 1.83e6,
        Hvap = 1.99e5,
        rho = 1596.5
    )

    g_dimer = tmo.Chemical(   # Couldn't find a chemical name so going with g_dimer
        'G_Dimer',
        default = True,
        search_db = False,
        MW = 362.42,
        phase = 'l',
        omega = 1.3966,
        Tb = 812.6,
        # Tc = 992,
        # Pc = 1.54e6,
        Hvap = 1.97e5,
        rho = 1497.3
    )

    s_oligomer = tmo.Chemical(
        'S_Oligomer',
        default = True,
        search_db = False,
        MW = 628.67,
        phase = 'l',
        omega = 0.78742,
        Tb = 921.4,
        # Tc = 1128,
        # Pc = 8.27e5
    )  

    g_oligomer = tmo.Chemical(    # Found little to no properties for G_Oligomer
        'G_Oligomer',
        default = True,
        search_db = False,
        MW = 540.65,
        phase = 'l'
    )


    # 4) Extend the base collection
    chems.extend([methanol, hydrogen, methane, activated_carbon, propylguaiacol, propylsyringol,syringaresinol,g_dimer,
                  s_oligomer, g_oligomer])

    chems.compile()  # Compiling all the chemicals to one string
    return chems


chems = create_chemicals()
bst.settings.set_thermo(chems) # Setting thermodynamic property pacakge for the chemicals


In [4]:
bst.settings.CEPCI = 541.7   # CEPCI for 2016, set to same as Bartling paper

In [5]:
# Defining poplar group based off composition given in Bartling et al Table S1

chems.define_group(
    name='Poplar',
    IDs=['Cellulose',   # Cellulase break this down to glucose
         'Xylan',       # Hemicellulose
         'Arabinan',    # Hemicellulose
         'Mannan',      # Hemicellulose
         'Galactan',    # Hemicellulose
         'Sucrose',   
         'Lignin',
         'Acetate',
         'Extract',     # Non structural component of biomass that dissolves in solvent/water 
         'Ash'],
    composition=[0.464,      # Dry wt composition, feed also has 20 wt% moisture content
                 0.134,
                 0.002,
                 0.037,
                 0.014,
                 0.001,
                 0.285,
                 0.035,
                 0.016,
                 0.012],
    wt=True
)


In [6]:
# Process conditions from Bartling et al 2021 unless specified otherwise


# Feed
biomass_flow = 2000                   # [dry metric tons per day], consistent with other TEA models 
biomass_moisture  = 0.2               # [%] 20% moisture

# RCF
rcf_temp = 200 + 273.15            # [K]
rcf_pressure = 60 * 1e5            # [bar]
solvolysis_tau = 2                 # [hr] residence time
hydrogenolysis_tau = 1             # [hr] residence time
meoh_h2o = 9                       # [ratio] Solvent : Water ratio
delignification = 0.7              # [%] 70% biomass delignified
methanol_decomp = 0.005            # [%] methanol decomposition rate (loss of solvent due to conversion into carbonaceous gases)

methanol_to_ch4 = 0.128/100        # [wt%] methanol lost as CH4. From https://pubs.rsc.org/en/content/articlelanding/2015/cc/c5cc04025f Table 1 where 0.08 mol% of methanol lost as CH4 for Ru/C catalyst
                                   # reactor was batch and hydrogen was fed at 3 MPa within the reactor, also biomass was birchwood so this might be different in my case

methanol_to_co = 0.364/100         # [wt%] methanol lost as CH4. From https://pubs.rsc.org/en/content/articlelanding/2015/cc/c5cc04025f Table 1 where 0.13 mol% of methanol lost as CH4 for Ru/C catalyst
                                   # reactor was batch and hydrogen was fed at 3 MPa within the reactor, also biomass was birchwood, so this might be different in my case



cellulose_retention = 0.9          # [%] 90% cellulose retained in biomass pulp after RCF
xylose_retention = 0.93            # [%] 93% xylose retained in biomass pulp after RCF
extractives_retention = 0          # [%] extractives retention in biomass from 10.1039/d1gc01591e Table S1 (no extractives in post-solvolysis poplar)
                                   # Also validated from SI Table S17 of Bartling et al where extractives total amount (kg) in RCF oil is similar to what is in the poplar feed 
acetate_retention = 0              # [%] acetate retention in biomass from 10.1039/d1gc01591e Table S1 (no acetate in post-solvolysis poplar)
cat_replace = 1                    # [/yr] catalyst replacement rate
methanol_to_biomass = 9            # [L/kg] from https://doi.org/10.1016/j.copbio.2018.12.005
                                   # Surprisingly, Bartling et al assumes a 9L/kg for a flow through
                                   # configuration which is very less.
hydrogen_amount = 10               # 10 L/min/dry kg biomass
catalyst_loading = 0.1             # 1:10 catalyst: dry biomass feed by wt 


# RCF oil composition
lignin_monomers = 0.5
lignin_dimers = 0.25
lignin_oligomers = 0.25 





In [7]:
poplar_in = bst.MultiStream('Poplar_In',
                s=[('Poplar', biomass_flow*1e3)], l=[('Water', biomass_moisture*biomass_flow*1e3)], 
                 phases = ('s','l'), units='kg/d')

In [8]:
meoh_in = bst.MultiStream('Methanol_In',
                          l = [('Methanol', methanol_to_biomass*biomass_flow*1e3*0.9),
                               ('Water', methanol_to_biomass*biomass_flow*1e3*(1/meoh_h2o)*0.9)],
                     phases = ('s','l','g'),   units = 'L/d')

# Set as a MultiStream to make it easier later on as this stream will also contain carbohydrate
# derivatives (solids) following solvolysis and hydrogen (gas) following hydrogenolysis

# both flows multipled by 0.9 to make the overall meoh_in to biomass liter to kg ratio as 90 which is a requirement

In [9]:
# Checks to ensure the correct methanol to biomass and methanol to water ratio is used
meoh_biomass_check = round((meoh_in.F_vol*1000)/(2e6/24),0)
methanol_water_check = round(meoh_in.ivol['Methanol']/meoh_in.ivol['Water'],0)

if meoh_biomass_check == methanol_to_biomass:
    print('Correct methanol to biomass ratio')

else:
    raise ValueError(f'Methanol to biomass ratio is {meoh_biomass_check} L/kg dry biomass \n It must be {methanol_to_biomass} L/kg dry biomass as specified')

if methanol_water_check == meoh_h2o:
    print('Correct methanol water ratio')

else:
    raise ValueError(f'Methanol water ratio is {methanol_water_check} \n It must be {meoh_h2o}')


Correct methanol to biomass ratio
Correct methanol water ratio


In [10]:
pump_try = bst.units.Pump(ins = meoh_in, P = 6e6)
heat_try = bst.units.HXutility(ins = pump_try-0, T = 463.15, rigorous = True)
pump_try.simulate()
heat_try.simulate()

In [11]:
heat_try.outs[0]

MultiStream: s2 from <HXutility: H1>
phases: ('g', 'l'), T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): (l) Water     4.15e+03
                    Methanol  1.66e+04


In [12]:
heat_try.outs[0].rho

605.5369549961845

In [13]:

# Solvolysis reaction to solubilize lignin from cell wall
solvolysis_rxn = bst.Reaction('Lignin,s -> SolubleLignin,l', reactant = 'Lignin', phases = 'sl', X = 0.7, basis = 'wt', correct_atomic_balance = False) 
# correct atomic balance set to False because we are working with pseudocomponents

methanol_decomposition_rxn = bst.ParallelReaction([
    bst.Reaction('Methanol,l -> Methane,g', reactant = 'Methanol', phases = 'slg', X = methanol_to_ch4 ,basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('Methanol,l -> CO,g', reactant = 'Methanol', phases = 'slg', X= methanol_to_co, basis = 'wt', correct_atomic_balance=False)
])


In [14]:
"""
References
-----------
    [1] Bartling, Andrew W., et al. 
    "Techno-economic analysis and life cycle assessment of a biorefinery utilizing
    reductive catalytic fractionation." Energy & Environmental Science 14.8 (2021): 4147-4168.

    [2] Anderson, Eric M., et al. 
    "Flowthrough reductive catalytic fractionation of biomass." Joule 1.3 (2017): 613-622.

    [3] Froment, Gilbert F., Kenneth B. Bischoff, and Juray De Wilde. 
    Chemical reactor analysis and design. Vol. 2. New York: Wiley, 1990.
-----------
"""

import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'SolvolysisReactor', 'RCF1'
    )

class SolvolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):

    """
    Plug flow reactor for solvolysis reaction, where a solvent is used to solubilize lignin present
    in plant cell wall
    

    In reality, lignin will also be partially depolymerized, but this phenomena is not captured in 
    current design but may be included in a future implementation
    Design based off [1],[2]. Pressure drop calculations from [3]

    By default,  current design supports 4 identical biomass beds, with 3 operational beds at any given time, 
    offset by 1 hour each. With a 3 hr reaction time + 1 hr turnaround, this gives near continuous throughput
    Example... t = 0 - 1 hr : bed 1, bed 2, bed 3 online, bed 4 cleaning, 
                t = 1 - 2 hr, bed 2, bed 3, bed 4 online, bed 1 cleaning
    Since at any given time, the complete throughput is mantained by 3 reactors, the total volumetric flow rate 
    of solvent is constant

    Parameters.....


    Examples
    


    Assumed: Extraction efficiency constant along the reaction residence time


    """



 

    auxiliary_unit_names = (
        'pump_1', 'heat_exchanger_1'
    )

    _N_ins = 2
    _N_outs = 2
    
    _units = {**PressureVessel._units,
              'Pressure drop': 'bar',
              'Batch time': 'hr',
              'Turnaround time': 'hr',
              'Residence time': 'hr',
              '# of beds online': "",
              '# of beds in service': "",
              'Total volume': 'm3',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 6 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 2

    #: Default cleaning and unloading time (hr).
    tau_0_default: float  = 1 # from https://doi.org/10.1039/D1EE01642C
    
    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default methanol decomposition (%)
    methanol_decomposition_default: float = 0.005 # from https://doi.org/10.1039/D1EE01642C

    # Default poplar bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default poplar diameter [m]
    poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            tau_0: Optional[float] = None,  
            superficial_velocity: Optional[float] = None,
            methanol_decomposition: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            poplar_diameter: Optional[float] = None,
            N_beds = 6, 
            *,
            reaction_1,
            reaction_2
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.tau_0 = self.tau_0_default if tau_0 is None else tau_0
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.methanol_decomposition = self.methanol_decomposition_default if methanol_decomposition is None else methanol_decomposition
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.poplar_diameter = self.poplar_diameter_default if poplar_diameter is None else poplar_diameter
        if N_beds != 6:
            raise ValueError('only 6 biomass beds are valid')
        self.reaction_1 = reaction_1
        self.reaction_2 = reaction_2
        self.N_beds = N_beds
        pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_1 = self.auxiliary('heat_exchanger_1', bst.HXutility, ins = pump_1.outs[0])


    def _size_bed(self):
        
        #### Reactor volume sizing ########

        cycle_time = self.tau + self.tau_0    # Total time for a cycle of operation (includes reaction + cleaning-loading-unloading)
        occupied_frac = 1 - self.void_frac    # ratio of occupied biomass fraction/total biomass bed
        working_vol = self.working_vol        # Working volume fraction for a reactor to allow suficient mass transfer by allowing enough contact time 
        solvent = self.ins[1]


        
        liquid_V_w = solvent.F_vol * cycle_time   # [m3] Total liquid working volume
        liquid_V_w_pr = liquid_V_w/self.N_beds    # [m3] Liquid working volume per reactor 
        V_actual = liquid_V_w_pr/(self.void_frac * working_vol)  # [m3]    void frac = vol. of solvent/total bed vol.
        total_volume = V_actual * self.N_beds

        ##### Reactor diameter and length ########
        u  = self.superficial_velocity                   # [m/s]

        N_operating = self.N_beds *(self.tau/cycle_time) # Number of beds operating at any given time
        Q_per_reactor_m3 = solvent.F_vol/N_operating     # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/(u*3600)
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual, total_volume        

        
    def _run(self):
        biomass, solvent = self.ins
        used_biomass, used_solvent = self.outs

        used_solvent.copy_like(solvent) 
        used_biomass.copy_like(biomass) 

        used_solvent.P = self.P # Outlet pressure is set to reactor pressure. Inlet pressure will be greater to account for pressure drop
        used_solvent.T = self.T # Since isothermal operation
        

        

        #ch4_produced = used_solvent.imass['Methanol']*methanol_to_ch4  
        # co_produced = used_solvent.imass['Methanol']*methanol_to_co




        # used_solvent.imass['l', 'Methanol'] = used_solvent.imass['l', 'Methanol']*(1-self.methanol_decomposition) # 0.5% methanol lost due to decomposition


        self.reaction_1(used_biomass) 
        self.reaction_2(used_solvent)
        

        solubilized_lignin = used_biomass.imass['SolubleLignin'] 
        used_solvent.imass['l', 'SolubleLignin'] += solubilized_lignin # Soluble lignin dissolves in solvent effluent stream 
        used_biomass.imass['l', 'SolubleLignin'] = 0 # No soluble lignin remaining in biomass (assuming 100% extraction efficiency)



        extractives = used_biomass.imass['Extract'] # From Table S1 https://www.rsc.org/suppdata/d1/gc/d1gc01591e/d1gc01591e1.pdf,
                                                        # it follows that the extractives component of poplar is 'extracted' in the solvent stream
        used_solvent.imass['l', 'Extract'] = (1-extractives_retention)*extractives
        used_biomass.imass['s', 'Extract'] = (extractives_retention)*extractives

        acetate = used_biomass.imass['s', 'Acetate']
        used_solvent.imass['l', 'Acetate'] =  acetate *(1-acetate_retention)  # Assuming acetate dissolves as acetic acid with methanol,
                                                                              # BioSTEAM Chemicals assumes same properties for acetic acid and acetate, otherwise is acetate was a pseudocomponent, it might have still stayed in solid phase
        used_biomass.imass['s', 'Acetate'] = acetate*acetate_retention


        cellulose_mass = used_biomass.imass['s', 'Cellulose']
        used_solvent.imass['l', 'Cellulose'] = cellulose_mass*(1-cellulose_retention) # Dissolved cellulose assumed to be in liquid phase as solution with solvent
        used_biomass.imass['s', 'Cellulose'] =  cellulose_mass*cellulose_retention
                                                               

        xylose_mass = used_biomass.imass['s', 'Xylose']
        used_solvent.imass['l', 'Xylose'] = xylose_mass * (1-xylose_retention) # Dissolved xylose assumed to be  liquid phase as solution with solvent
        used_biomass.imass['s', 'Xylose'] = xylose_mass * xylose_retention
        





    def _calculate_pressure_drop(self, bed_length):

        D = self.poplar_diameter # [m] poplar particle diameter
        rho = self.outs[1].rho  # [kg/m3] 
        mu = self.outs[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        epsilon = self.void_frac # Void fraction 
        u = self.superficial_velocity # [m/s] superficial velocity  

        
        
        Re = (D*rho*u)/mu
        if Re/(1-epsilon) < 500: # Erun equation
            f = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
            dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5             # [bar] 1e-5 converts Pa to bar
        elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
            f = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
            dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
        else: # Hicks equation which fits in Wentz and Thodos results for very high Re
            f = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
            dP = (f * ((rho*(u**2))/D) * bed_length)*1e-5
        return dP

        

    def _design(self):
        length, diameter, V_actual, total_volume = self._size_bed()   # Calling size bed function to determine diameter and length 
        

        cycle_time = self.tau + self.tau_0

        self.set_design_result('Diameter', 'ft', diameter)  
        self.set_design_result('Length', 'ft', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Total volume', 'm3', total_volume)
        self.set_design_result('Residence time', 'hr', self.tau)
        self.set_design_result('Turnaround time', 'hr', self.tau_0)
        self.set_design_result('Batch time', 'hr', cycle_time)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        pressure_drop = self._calculate_pressure_drop(length)                  
        
        self.set_design_result('Pressure drop', 'bar', pressure_drop)
        self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        self.pump_1.simulate()

        self.heat_exchanger_1.T = self.T
        self.heat_exchanger_1.simulate()
        






    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )

        self.parallel['self'] = self.N_beds # Used to create multiple of the same beds
        # self.parallel['pump_1'] = 1


        
       
        """
        ---------

        - Duty requirements are too high - need to check thermodynamics of the solvolysis step
        and modify requirements if needed
          
        - Might need to alter data on working volume, void fraction of poplar, poplar particle size,
        for more realistic modeling

        ----------

        """



        

        


In [15]:
D = 0.004                   # [m] poplar particle diameter
rho = 600                   # [kg/m3] 
mu = 9.71e-05               # [Pa s] methanol water viscosity 
epsilon = 0.3               # void fraction 
u = 0.003                     # [m/s] superficial velocity  

bed_length = 150

Re = (D*rho*u)/mu
if Re/(1-epsilon) < 500: # Erun equation
    f = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
    dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
    f = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
    dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
else: # Hicks equation which fits in Wentz and Thodos results for very high Re
    f = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
    dP = (f * ((rho*(u**2))/D) * bed_length)*1e-5



In [16]:
# Created as a check to see if logic inside the solvolysis reactor class is working
N_beds = 6
tau = 2
cycle_time = 3
N_operating  = 4
u = 0.003
void_frac = 0.5
working_vol = 0.8

total_sol_V = meoh_in.F_vol * cycle_time         # [m3]
total_sol_V_per_reactor = total_sol_V/N_beds   # [m3]
actual_V_per_reactor = total_sol_V_per_reactor/(void_frac*working_vol)

N_operating = N_beds *(tau/cycle_time) # Number of beds operating at any given time
Q_per_reactor_m3 = meoh_in.F_vol/N_operating     # [m3/hr] Volumetric flow rate processed by any reactor
A  = Q_per_reactor_m3/(u*3600)                   # [m2]
diameter = 2 * (A/np.pi) ** 0.5                  # [m]
length = actual_V_per_reactor/A                  # [m]

In [17]:
print(f'Actual reactor volume is: {actual_V_per_reactor} m3')

Actual reactor volume is: 937.5000000000001 m3


In [18]:
meoh_in

MultiStream: Methanol_In to <Pump: P1>
phases: ('g', 'l', 's'), T: 298.15 K, P: 101325 Pa
flow (kmol/hr): (l) Water     4.15e+03
                    Methanol  1.66e+04


In [19]:
solvolysis_reactor = SolvolysisReactor(ins = (poplar_in, meoh_in), outs = ('Carbohydrate_Pulp', 'Solvolysis_Liquor'),
                                    T = rcf_temp,
                                    P = rcf_pressure,
                                    tau = solvolysis_tau, 
                                    N_beds = 6,
                                    void_frac= 0.5,
                                    superficial_velocity = 0.001, 
                                    reaction_1 = solvolysis_rxn,
                                    reaction_2 = methanol_decomposition_rxn
                                    )
solvolysis_reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Methanol_In from P1; Methanol_In is now docked at R1
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\3459856161.py:284: CostWarning: <SolvolysisReactor: R1> Vertical vessel weight (3.747e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\3459856161.py:284: CostWarning: <SolvolysisReactor: R1> Vertical vessel length (59.06 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'SolvolysisReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}'

In [20]:
solvolysis_reactor

SolvolysisReactor: R1
ins...
[0] Poplar_In  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     156
                        Cellulose  238
                        Xylan      84.5
                        Arabinan   1.26
                        Mannan     19
                        Galactan   7.2
[1] Methanol_In  
    phases: ('g', 'l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water     4.15e+03
                        Methanol  1.66e+04
outs...
[0] Carbohydrate_Pulp  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Ash        1e+03
                        Lignin     46.8
                        Cellulose  215
                        Xylan

In [21]:
solvolysis_reactor.results()

Solvolysis reactor                                         Units        R1
Electricity         Power                                     kW  9.19e+03
                    Cost                                  USD/hr       719
High pressure steam Duty                                   kJ/hr  2.67e+09
                    Flow                                 kmol/hr  8.29e+04
                    Cost                                  USD/hr  2.63e+04
Design              Diameter                                  ft      26.7
                    Length                                    ft      59.1
                    Reactor volume                            m3       938
                    Total volume                              m3  5.63e+03
                    Residence time                            hr         2
                    Turnaround time                           hr         1
                    Batch time                                hr         3
                    Vessel type                                   Vertical
                    Weight                                    lb  3.75e+06
                    Wall thickness                            in      12.5
                    Pressure drop                            bar  0.000526
Purchase cost       Vertical pressure vessel (x6)            USD  4.64e+07
                    Platform and ladders (x6)                USD  4.77e+05
                    Pump 1 - Pump (x6)                       USD  2.89e+06
                    Pump 1 - Motor (x30)                     USD  1.57e+06
                    Heat exchanger 1 - Floating head...      USD  1.97e+06
Total purchase cost                                          USD  5.33e+07
Utility cost                                              USD/hr   2.7e+04

In [22]:
carbohydrate_derivates  = ((solvolysis_reactor.outs[1].imass['Acetate']
  + solvolysis_reactor.outs[1].imass['Cellulose'] + solvolysis_reactor.outs[1].imass['Xylan'])*24*330)/1e6

print(f'Carbohydrate derivates in solvent due to dissolution constitute {carbohydrate_derivates} MMkg/yr \n In Bartling et al, it is 54.2 MM kg/yr')

Carbohydrate derivates in solvent due to dissolution constitute 53.724 MMkg/yr 
 In Bartling et al, it is 54.2 MM kg/yr


In [23]:
hydrogen_in = bst.Stream('Hydrogen_In', 
                         Hydrogen = (10)*(2e6/24/60),   # Anderson et al, had MeOH/H2 feeds as 1:100 vol:vol ratio
                         units = 'L/min',
                         T = 473.15,
                         P = 6e6,
                         phase = 'g'
)



In [24]:
heaterr = bst.units.HXutility(ins = hydrogen_in, T = 463.15, rigorous = True)
pumpp = bst.units.Pump(ins = heaterr-0, P = 6e6)
heaterr.simulate()
pumpp.simulate()

In [25]:
pumpp.results()

Pump                              Units               P2
Electricity         Power            kW             38.5
                    Cost         USD/hr             3.01
Design              Type                 MeteringPlunger
                    Ideal power      hp             3.85
                    Flow rate       gpm              449
                    Efficiency                     0.597
                    Power            hp             6.45
                    Head             ft         1.87e+04
                    Motor size       hp              7.5
Purchase cost       Pump (x8)       USD         4.46e+04
                    Motor (x64)     USD         3.96e+03
Total purchase cost                 USD         4.86e+04
Utility cost                     USD/hr             3.01

In [26]:
pumpp.outs[0].F_vol

816.166636361352

In [27]:
1995.9104575780248/75000.00000000001


0.02661213943437366

In [28]:
((hydrogen_in.F_mass*24)/2e6) * 0.02661213943437366

0.0007909159544009323

In [29]:
h2_flow = 10*(2e6/(24*60))

In [30]:
chems['Hydrogen'].rho(T = 4.15, P = 1e6, phase = 'g')

0.0004995845346753258

In [31]:
(h2_flow/1000)*60

833.3333333333334

In [32]:
hydrogenolysis = bst.ParallelReaction([
    bst.Reaction('SolubleLignin,l ->Propylguaiacol,l', reactant = 'SolubleLignin', phases = 'slg', X = 0.25, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l ->Propylsyringol,l', reactant = 'SolubleLignin', phases = 'slg', X = 0.25, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> Syringaresinol,l', reactant= 'SolubleLignin',phases = 'slg', X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> G_Dimer,l', reactant= 'SolubleLignin', phases = 'slg',X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> S_Oligomer,l', reactant= 'SolubleLignin',phases = 'slg', X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> G_Oligomer,l', reactant= 'SolubleLignin', phases = 'slg',X = 0.125, basis = 'wt', correct_atomic_balance=False),
])

In [33]:
import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'HydrogenolysisReactor', 'RCF2'
    )

class HydrogenolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):


    auxiliary_unit_names = (
        'heat_exchanger_2'
    )

    _N_ins = 2
    _N_outs = 1
    
    _units = {**PressureVessel._units,
              # 'Pressure drop': 'bar',
              'Residence time': 'hr',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 6 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 3

    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default catalyst bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default catayyst diameter [m]
    # poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            superficial_velocity: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.reaction = reaction
        #pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_2 = self.auxiliary('heat_exchanger_2', bst.HXutility)


    def _size_bed(self):

        #### Reactor volume sizing ########

        residence_time = self.tau
        occupied_frac = 1 - self.void_frac  
        working_vol = self.working_vol        
        influent = self.ins[0]
        liquid_V_w = influent.F_vol * residence_time # [m3] Total liquid working volume
        V_actual = liquid_V_w/(occupied_frac * working_vol)  # [m3]

        ##### Reactor diameter and length ########
        u  = self.superficial_velocity

        Q_per_reactor_m3 = influent.F_vol    # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/u
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual
        
    def _run(self):
        solvent, hydrogen = self.ins
        effluent, = self.outs

        

        effluent.copy_like(solvent)
        self.reaction(effluent)

        h2 = hydrogen.imass['Hydrogen']

        effluent.imass['g', 'Hydrogen'] = h2

        effluent.T = self.T # Assuming isothermal operation
        effluent.P = self.P # Assuming no P drop




    #def _calculate_pressure_drop(self):
        # NOT OPERATIONAL FOR HYDROGENOLYSIS REACTOR

        #D = self.poplar_diameter # [m] poplar particle diameter
        #rho_solv = self.ins[1].rho  # [kg/m3] methanol water density
        #mu = self.ins[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        #epsilon = self.void_frac # Void fraction 
        #u = self.superficial_velocity # [m/s] superficial velocity          
        
        #Re = (D*rho_solv*u)/mu
        #if Re/(1-epsilon) < 500: # Erun equation
        #    dP = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
        #elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
        #    dP = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
        #else: # Hicks equation which fits in Wentz and Thodos results for very high Re
        #    dP = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
        #return dP

        

    def _design(self):
        diameter, length, V_actual = self._size_bed()   # Calling size bed function to determine diameter and length 


        self.set_design_result('Diameter', 'ft', diameter)  
        self.set_design_result('Length', 'ft', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Residence time', 'hr', self.tau)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        #pressure_drop = self._calculate_pressure_drop()                  
        
        #self.set_design_result('Pressure drop', 'bar', pressure_drop)
        #self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        #self.pump_1.simulate()

        #self.heat_exchanger_2.T = self.T
        #self.heat_exchanger_2.simulate()
        





        

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )



        
        """



        """



        


In [34]:
hydrogenolysis_reactor = HydrogenolysisReactor(ins = (solvolysis_reactor.outs[1], hydrogen_in), reaction = hydrogenolysis)
hydrogenolysis_reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Hydrogen_In from H2; Hydrogen_In is now docked at R2
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\287338450.py:160: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel weight (2.661e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\287338450.py:160: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel length (122.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'HydrogenolysisReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item

In [35]:
hydrogenolysis_reactor.results()

Hydrogenolysis reactor                         Units        R2
Design              Diameter                      ft      17.6
                    Length                        ft       122
                    Reactor volume                m3  5.86e+03
                    Residence time                hr         3
                    Vessel type                       Vertical
                    Weight                        lb  2.66e+06
                    Wall thickness                in      8.27
Purchase cost       Vertical pressure vessel     USD  5.74e+06
                    Platform and ladders         USD  9.76e+04
Total purchase cost                              USD  5.84e+06
Utility cost                                  USD/hr         0

In [36]:
hydrogenolysis_reactor

HydrogenolysisReactor: R2
ins...
[0] Solvolysis_Liquor  from  SolvolysisReactor-R1
    phases: ('g', 'l', 's'), T: 473.15 K, P: 6e+06 Pa
    flow (kmol/hr): (g) CO             69
                        Methane        42.4
                    (l) Water          4.15e+03
                        Extract        7.4
                        Acetate        48.6
                        SolubleLignin  109
                        Cellulose      23.8
                        Methanol       1.65e+04
[1] Hydrogen_In  
    phase: 'g', T: 473.15 K, P: 6e+06 Pa
    flow (kmol/hr): Hydrogen  1.23e+03
outs...
[0] s5  
    phases: ('g', 'l', 's'), T: 463.15 K, P: 6e+06 Pa
    flow (kmol/hr): (g) CO              69
                        Hydrogen        1.23e+03
                        Methane         42.4
                    (l) Water           4.15e+03
                        Extract         7.4
                        Acetate         48.6
                        Cellulose       23.8
                  

In [37]:
hydrogenolysis_reactor-0

MultiStream: s5 from <HydrogenolysisReactor: R2>
phases: ('g', 'l', 's'), T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): (g) CO              69
                    Hydrogen        1.23e+03
                    Methane         42.4
                (l) Water           4.15e+03
                    Extract         7.4
                    Acetate         48.6
                    Cellulose       23.8
                    Methanol        1.65e+04
                    Propylguaiacol  25
                    Propylsyringol  21.2
                    Syringaresinol  4.97
                    G_Dimer         5.73
                    S_Oligomer      3.31
                    G_Oligomer      3.84


In [38]:
R102 = bst.units.Flash(ins = hydrogenolysis_reactor-0, T = 320, P = 5e5)
R102.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Methane has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)


In [39]:
R102

Flash: F1
ins...
[0] s5  from  HydrogenolysisReactor-R2
    phases: ('g', 'l'), T: 453.56 K, P: 6e+06 Pa
    flow (kmol/hr): (g) Water           112
                        CO              69
                        Acetate         0.18
                        Methanol        843
                        Hydrogen        1.23e+03
                        Methane         42.4
                    (l) Water           4.04e+03
                        Extract         7.4
                        Acetate         48.4
                        Cellulose       23.8
                        Methanol        1.56e+04
                        Propylguaiacol  25
                        Propylsyringol  21.2
                        Syringaresinol  4.97
                        G_Dimer         5.73
                        S_Oligomer      3.31
                        G_Oligomer      3.84
outs...
[0] s6  
    phase: 'g', T: 320 K, P: 500000 Pa
    flow (kmol/hr): Water     8.9
                    CO        69
  

In [40]:
R102.results()

Flash                                                 Units            F1
Cooling water       Duty                              kJ/hr     -3.23e+08
                    Flow                            kmol/hr       2.2e+05
                    Cost                             USD/hr           107
Design              Vessel type                                Horizontal
                    Length                               ft          27.2
                    Diameter                             ft            18
                    Weight                               lb      9.11e+04
                    Wall thickness                       in         0.872
                    Vessel material                          Carbon steel
Purchase cost       Horizontal pressure vessel          USD      1.09e+05
                    Platform and ladders                USD      3.91e+03
                    Heat exchanger - Floating head      USD       4.7e+05
Total purchase cost                                     USD      5.83e+05
Utility cost                                         USD/hr           107

In [41]:
R102.outs[1].show(flow = 'kg/hr', composition = True)

Stream: s7 from <Flash: F1>
phase: 'l', T: 320 K, P: 500000 Pa
composition (%): Water           12
                 Extract         0.214
                 Acetate         0.467
                 Cellulose       0.62
                 Methanol        84.1
                 Methane         0.000719
                 Propylguaiacol  0.666
                 Propylsyringol  0.666
                 Syringaresinol  0.333
                 G_Dimer         0.333
                 S_Oligomer      0.333
                 G_Oligomer      0.333
                 --------------  6.24e+05 kg/hr


In [42]:
R102-0

Stream: s6 from <Flash: F1>
phase: 'g', T: 320 K, P: 500000 Pa
flow (kmol/hr): Water     8.9
                CO        69
                Acetate   0.0458
                Methanol  115
                Hydrogen  1.23e+03
                Methane   42.1


In [43]:
MockPSA = bst.units.Splitter('Mock_PSA', ins = R102-0, outs = ('Hydrogen_Recycle', 'Purge_Gases'), split = {'Hydrogen':1})
MockPSA.simulate()

In [44]:
crude_distillation = bst.units.BinaryDistillation(ins = R102-1, 
                                                LHK = ('Methanol', 'Water'),
                                                  Lr = 0.9995,  Hr = 1-0.967, P = 101325,
                                                  k = 2, partial_condenser = True)
crude_distillation.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Methane has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\dew_point.py:129: RuntimeWarning: Methane has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)


In [45]:
crude_distillation.outs[0].show(flow = 'L/hr', composition = True)

Stream: s8 from <BinaryDistillation: D1>
phase: 'g', T: 345.47 K, P: 101325 Pa
composition (%): Water     19.7
                 Methanol  80.3
                 Methane   0.0014
                 --------  5.66e+08 L/hr


In [46]:
meoh_purifier_col = bst.units.BinaryDistillation(ins = crude_distillation-0,
                                                 outs = ('Methanol_Recycle', 'To_WW_Treatment'),
                                                LHK = ('Methanol', 'Water'),
                                                 y_top = 0.9, x_bot = 0.001, P = 101325,
                                                 k = 2)
meoh_purifier_col.simulate()



In [47]:
meoh_purifier_col.outs[0].show(flow = 'L/hr', composition = True)
# Composition as expected and required

Stream: Methanol_Recycle from <BinaryDistillation: D2>
phase: 'g', T: 341.48 K, P: 101325 Pa
composition (%): Water     10
                 Methanol  90
                 Methane   0.00157
                 --------  4.98e+08 L/hr


In [48]:
crude_distillation.outs[0].show(flow = 'L/hr', composition = True)

Stream: s8 from <BinaryDistillation: D1> to <BinaryDistillation: D2>
phase: 'g', T: 345.47 K, P: 101325 Pa
composition (%): Water     19.7
                 Methanol  80.3
                 Methane   0.0014
                 --------  5.66e+08 L/hr


In [49]:
crude_distillation.results()

Distillation Column                                        Units        D1
Electricity         Power                                     kW      24.3
                    Cost                                  USD/hr       1.9
Cooling water       Duty                                   kJ/hr -1.57e+07
                    Flow                                 kmol/hr  1.07e+04
                    Cost                                  USD/hr      5.24
Low pressure steam  Duty                                   kJ/hr  8.47e+08
                    Flow                                 kmol/hr  2.19e+04
                    Cost                                  USD/hr   5.2e+03
Design              Theoretical feed stage                               1
                    Theoretical stages                                   4
                    Minimum reflux                         Ratio      0.01
                    Reflux                                 Ratio      0.02
                    Actual stages                                       11
                    Height                                    ft      27.3
                    Diameter                                  ft      32.3
                    Wall thickness                            in      1.25
                    Weight                                    lb  2.77e+05
Purchase cost       Trays                                    USD  8.08e+05
                    Tower                                    USD  5.68e+05
                    Platform and ladders                     USD  4.17e+04
                    Condenser - Floating head                USD  4.92e+04
                    Reflux drum - Vertical pressure ...      USD  4.12e+05
                    Reflux drum - Platform and ladders       USD   4.4e+04
                    Pump - Pump                              USD  1.54e+04
                    Pump - Motor                             USD  1.36e+03
                    Reboiler - Floating head                 USD  4.83e+05
Total purchase cost                                          USD  2.42e+06
Utility cost                                              USD/hr  5.21e+03

In [57]:
water_remover = bst.units.Flash(ins = crude_distillation-1, outs = ('To WW_Treatment_2', 
                                                                    'RCF-Oil'), T = 400, P = 101325)
water_remover.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet s9 from F2; s9 is now docked at F3
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\_stream.py:398: RuntimeWarning: <Stream: RCF-Oil> has been replaced in registry
  self._register(ID)


In [51]:
#rcf_prod = tmo.MultiStream(phases = ('s','l','g'))
#rcf_prod.mix_from((R101-1, crude_distillation-1), energy_balance = False) # Setting energy balance equal true makes it unfeasible because of the pseudocomponents and their lack of thermo properties

In [52]:
#rcf_prod.show(flow = 'kg/yr')

In [53]:
# rcf_prod.show(composition = True, flow = 'kg/hr')

In [54]:
rcf_system = bst.System('RCF_System', path = (solvolysis_reactor, hydrogenolysis_reactor, 
                                              R102, MockPSA, crude_distillation, meoh_purifier_col,
                                              water_remover))
rcf_system.simulate()

C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\3459856161.py:284: CostWarning: <SolvolysisReactor: R1> Vertical vessel weight (3.747e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\3459856161.py:284: CostWarning: <SolvolysisReactor: R1> Vertical vessel length (59.06 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\287338450.py:160: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel weight (2.661e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_44956\287338450.py:160: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel length (122.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(


In [55]:
rcf_system

System: RCF_System
ins...
[0] Poplar_In  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     156
                        Cellulose  238
                        Xylan      84.5
                        Arabinan   1.26
                        Mannan     19
                        Galactan   7.2
[1] Methanol_In  
    phases: ('g', 'l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water     4.15e+03
                        Methanol  1.66e+04
[2] Hydrogen_In  
    phase: 'g', T: 473.15 K, P: 6e+06 Pa
    flow (kmol/hr): Hydrogen  1.23e+03
outs...
[0] Carbohydrate_Pulp  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Ash        1e+03
             

In [ ]:
rcf_system.diagram(file = 'rcfv3', format = 'png')